In [ ]:
import os
import json
import yaml

import numpy as np
import pandas as pd

from decode_trf import delivery_data_from_logfile

from pymedphys.level1.msqconnect import mosaiq_connect
from pymedphys.level1.configutilities import get_index, get_data_directory, get_filepath, get_gantry_tolerance
from pymedphys.level1.mudensity import find_relevant_control_points, calc_mu_density
from pymedphys.level2.msqdelivery import multi_fetch_and_verify_mosaiq
from pymedphys.level3.analyselogfiles import plot_results, calc_comparison
from pymedphys.level4.comparebygantry import (
    get_mappings, get_comparisons_byfield, get_logfile_delivery_data_bygantry,
    get_logfile_mu_density_bygantry, get_mosaiq_delivery_data_bygantry,
    get_mosaiq_mu_density_bygantry, get_comparison_results, extract_angle_from_delivery_data,
    get_field_id_from_logfile_group, calc_mosaiq_mu_density_bygantry, 
    calc_logfile_mu_density_bygantry
)

In [ ]:
with open('../config.json') as config_file:
    config = json.load(config_file)

In [ ]:
index = get_index(config)

In [ ]:
data_directory = get_data_directory(config)

cache_filepath = os.path.join(data_directory, 'cache', 'dmlc_comparison.yaml')
cache_scratch_filepath = os.path.join(data_directory, 'cache', 'dmlc_comparison_scratch.yaml')

with open(cache_filepath, 'r') as cache_file:
    cache = yaml.load(cache_file)

In [ ]:
patient_ids = list(cache.keys())
len(patient_ids)

In [ ]:
data = []

for patient_id in patient_ids:
    for byfield in cache[patient_id]:
        for angle in cache[patient_id][byfield]:
            comparison = cache[patient_id][byfield][angle]['median']
            file_hashes = cache[patient_id][byfield][angle]['median_filehash_group']
            data.append([
                patient_id, byfield, angle, comparison, file_hashes
            ])
            
comparisons_table = pd.DataFrame(
    columns=[
        'patient_id', 'field_id',
        'gantry_angle', 'comparison',
        'file_hashes'
    ],
    data=data
)

comparisons_table = comparisons_table.sort_values('comparison', ascending=False)
top_ten = comparisons_table.iloc[0:10]
top_ten

In [ ]:
len(comparisons_table)

In [ ]:
logfile_group = comparisons_table.iloc[0]['file_hashes']
logfile_group

In [ ]:
gantry_angle = comparisons_table.iloc[0]['gantry_angle']
gantry_angle

In [ ]:
field_id = get_field_id_from_logfile_group(index, logfile_group)

In [ ]:
with mosaiq_connect('msqsql') as cursor:
    mosaiq_mu_density = calc_mosaiq_mu_density_bygantry(cursor, field_id, gantry_angle)

In [ ]:
logfile_mu_density = calc_logfile_mu_density_bygantry(index, config, logfile_group, gantry_angle)

In [ ]:
grid_xx = logfile_mu_density[0]
grid_yy = logfile_mu_density[1]

comparison = calc_comparison(logfile_mu_density[2], mosaiq_mu_density[2])

print(comparison)
plot_results(grid_xx, grid_yy, logfile_mu_density[2], mosaiq_mu_density[2])